In [2]:
pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip install tradermade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 78 kB 3.8 MB/s 
  Created wheel for tradermade: filename=tradermade-0.5.0-py3-none-any.whl size=4883 sha256=29c38bc1892b2f52223363dafd98041e3e8a00d9bba9cd6069cf11ab6a406d30
  Stored in directory: /root/.cache/pip/wheels/c7/c9/1b/8b76044329aadd20ac3a4ebb0f8c1189d21d1fe743b47c312a
Successfully built tradermade
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [9]:
from dotenv import load_dotenv
import os
import pandas as pd
from pathlib import Path
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import tradermade as tm

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv()

True

In [6]:
# Retrieve API key and store as Python variable
api_key = os.getenv("TRADERMADE_API_KEY")
type(api_key)

str

In [10]:
# set api key
tm.set_rest_api_key(api_key)

# Gain Forex close price time series data from API

In [11]:
# get timeseries data
import pandas as pd
df = pd.DataFrame()
for i in range(2012, 2022):  
    x = tm.timeseries(currency='AUDUSD', start=str(i)+"-12-01",fields=["open", "high", "low","close"], end=str(i+1)+"-11-30")
    df = df.append(x)
df = df.drop_duplicates()
df
# returns timeseries data for the currency requested interval is daily, hourly, minute - fields parameter is optional (you can select ["close"] if you just want close prices)

,date,open,high,low,close
0,2012-12-03,1.04280,1.04450,1.03940,1.04220
1,2012-12-04,1.04200,1.04840,1.04150,1.04710
2,2012-12-05,1.04720,1.04840,1.04430,1.04560
3,2012-12-06,1.04560,1.05140,1.04420,1.04870
4,2012-12-07,1.04870,1.04960,1.04610,1.04890
...,...,...,...,...,...
256,2022-11-24,0.67324,0.67785,0.67272,0.67646
257,2022-11-25,0.67646,0.67812,0.67207,0.67524
258,2022-11-28,0.67215,0.67306,0.66420,0.66524
259,2022-11-29,0.66500,0.67489,0.66404,0.66879


In [12]:
df.index = pd.to_datetime(df['date'])

In [13]:
df = df.drop(columns = "date")
df

,open,high,low,close
date,,,,
2012-12-03,1.04280,1.04450,1.03940,1.04220
2012-12-04,1.04200,1.04840,1.04150,1.04710
2012-12-05,1.04720,1.04840,1.04430,1.04560
2012-12-06,1.04560,1.05140,1.04420,1.04870
2012-12-07,1.04870,1.04960,1.04610,1.04890
...,...,...,...,...
2022-11-24,0.67324,0.67785,0.67272,0.67646
2022-11-25,0.67646,0.67812,0.67207,0.67524
2022-11-28,0.67215,0.67306,0.66420,0.66524


In [14]:
df_monthly = df.groupby(pd.Grouper(freq='M')).mean()

In [15]:
close_monthly = df_monthly.drop(columns=["open", "high", "low"])

In [16]:
close_monthly_reset_index = close_monthly.reset_index()

In [17]:
close_monthly_reset_index

,date,close
0,2012-12-31,1.046019
1,2013-01-31,1.050035
2,2013-02-28,1.030860
3,2013-03-31,1.034862
4,2013-04-30,1.037859
...,...,...
115,2022-07-31,0.686085
116,2022-08-31,0.695626
117,2022-09-30,0.667321
118,2022-10-31,0.636295


In [18]:
close_monthly_reset_index.to_csv('close_monthly_reset_index.csv', encoding='utf-8', index=True)

# Read US unemployment rate data from csv file to a dataframe, convert the data into time series and slice the data from Dec 2012 to Nov 2022 

In [21]:
from google.colab import files
uploaded = files.upload()

Saving US-Unemployment rate-2012 to 2022.csv to US-Unemployment rate-2012 to 2022.csv


In [22]:
# read us unemployment rate csv file into dataframe
path_str_us = "US-Unemployment rate-2012 to 2022.csv"
path_us = Path(path_str_us)
data_us = pd.read_csv(path_us, parse_dates = True,infer_datetime_format = True)
data_us

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2012,8.3,8.3,8.2,8.2,8.2,8.2,8.2,8.1,7.8,7.8,7.7,7.9
1,2013,8.0,7.7,7.5,7.6,7.5,7.5,7.3,7.2,7.2,7.2,6.9,6.7
2,2014,6.6,6.7,6.7,6.2,6.3,6.1,6.2,6.1,5.9,5.7,5.8,5.6
3,2015,5.7,5.5,5.4,5.4,5.6,5.3,5.2,5.1,5.0,5.0,5.1,5.0
4,2016,4.8,4.9,5.0,5.1,4.8,4.9,4.8,4.9,5.0,4.9,4.7,4.7
5,2017,4.7,4.6,4.4,4.4,4.4,4.3,4.3,4.4,4.3,4.2,4.2,4.1
6,2018,4.0,4.1,4.0,4.0,3.8,4.0,3.8,3.8,3.7,3.8,3.8,3.9
7,2019,4.0,3.8,3.8,3.6,3.6,3.6,3.7,3.7,3.5,3.6,3.6,3.6
8,2020,3.5,3.5,4.4,14.7,13.2,11.0,10.2,8.4,7.9,6.9,6.7,6.7
9,2021,6.4,6.2,6.0,6.0,5.8,5.9,5.4,5.2,4.7,4.6,4.2,3.9


In [23]:
# convert to 1 dimension time series data
new_data_us=data_us.set_index('Year')
df1_us=new_data_us.stack()
df1_us

Year     
2012  Jan    8.3
      Feb    8.3
      Mar    8.2
      Apr    8.2
      May    8.2
            ... 
2022  Jul    3.5
      Aug    3.7
      Sep    3.5
      Oct    3.7
      Nov    3.7
Length: 131, dtype: float64

In [24]:
df2_us=df1_us.reset_index()
df2_us

,Year,level_1,0
0,2012,Jan,8.3
1,2012,Feb,8.3
2,2012,Mar,8.2
3,2012,Apr,8.2
4,2012,May,8.2
...,...,...,...
126,2022,Jul,3.5
127,2022,Aug,3.7
128,2022,Sep,3.5
129,2022,Oct,3.7


In [25]:
# Rename columns' name
df2_us.columns = ['Year','Month','us_rate']
df2_us

,Year,Month,us_rate
0,2012,Jan,8.3
1,2012,Feb,8.3
2,2012,Mar,8.2
3,2012,Apr,8.2
4,2012,May,8.2
...,...,...,...
126,2022,Jul,3.5
127,2022,Aug,3.7
128,2022,Sep,3.5
129,2022,Oct,3.7


In [26]:
# Slice the data to Dec 2012 to Nov 2022 period
df3_us= df2_us[11:]
df3_us=df3_us.reset_index()
df3_us

,index,Year,Month,us_rate
0,11,2012,Dec,7.9
1,12,2013,Jan,8.0
2,13,2013,Feb,7.7
3,14,2013,Mar,7.5
4,15,2013,Apr,7.6
...,...,...,...,...
115,126,2022,Jul,3.5
116,127,2022,Aug,3.7
117,128,2022,Sep,3.5
118,129,2022,Oct,3.7


In [27]:
# Check if there is no null data in the dataframe
df3_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    120 non-null    int64  
 1   Year     120 non-null    int64  
 2   Month    120 non-null    object 
 3   us_rate  120 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.9+ KB


In [28]:
df_us=df3_us.drop(columns=['index','Year','Month'])
df_us

,us_rate
0,7.9
1,8.0
2,7.7
3,7.5
4,7.6
...,...
115,3.5
116,3.7
117,3.5
118,3.7


# Read AU unemployment rate data from csv file to a dataframe, slice the data from Dec 2012 to Nov 2022 

In [29]:
from google.colab import files
uploaded = files.upload()

Saving AU-Unemployment rate-2012 to 2022.csv to AU-Unemployment rate-2012 to 2022.csv


In [30]:
# Read au unemployment data from csv file to dataframe
path_str_au = "AU-Unemployment rate-2012 to 2022.csv"
path_au = Path(path_str_au)
data_au = pd.read_csv(path_au, parse_dates = True,infer_datetime_format = True)
data_au

,Unnamed: 0,Trend (%),Seasonally adjusted (%)
0,Nov-12,5.4,5.3
1,Dec-12,5.4,5.4
2,Jan-13,5.4,5.4
3,Feb-13,5.5,5.4
4,Mar-13,5.6,5.6
...,...,...,...
120,Nov-22,3.5,3.4
121,NaN,NaN,NaN
122,Large month-to-month changes occurred during t...,NaN,NaN
123,NaN,NaN,NaN


In [31]:
# Slice the data from Dec 2012
data_au=data_au[1:121]
data_au

,Unnamed: 0,Trend (%),Seasonally adjusted (%)
1,Dec-12,5.4,5.4
2,Jan-13,5.4,5.4
3,Feb-13,5.5,5.4
4,Mar-13,5.6,5.6
5,Apr-13,5.6,5.6
...,...,...,...
116,Jul-22,3.6,3.5
117,Aug-22,3.5,3.5
118,Sep-22,3.5,3.6
119,Oct-22,3.5,3.4


In [32]:
# Drop null data and slice the data from Dec 2012 to Nov 2022
data_au=data_au.reset_index()
data_au

,index,Unnamed: 0,Trend (%),Seasonally adjusted (%)
0,1,Dec-12,5.4,5.4
1,2,Jan-13,5.4,5.4
2,3,Feb-13,5.5,5.4
3,4,Mar-13,5.6,5.6
4,5,Apr-13,5.6,5.6
...,...,...,...,...
115,116,Jul-22,3.6,3.5
116,117,Aug-22,3.5,3.5
117,118,Sep-22,3.5,3.6
118,119,Oct-22,3.5,3.4


In [33]:
# Check if there is no null data in the dataframe, since the data in trend column has null data, we will use seasonally adjusted data for the unemployment rates
data_au.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    120 non-null    int64  
 1   Unnamed: 0               120 non-null    object 
 2   Trend (%)                96 non-null     float64
 3   Seasonally adjusted (%)  120 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.9+ KB


In [34]:
# Rename columns' name
data_au_rate=data_au.drop(columns=['index','Unnamed: 0','Trend (%)'])

data_au_rate.columns= ['au_rate']
data_au_rate

,au_rate
0,5.4
1,5.4
2,5.4
3,5.6
4,5.6
...,...
115,3.5
116,3.5
117,3.6
118,3.4


In [35]:
us_au_unemployment_df = pd.concat([data_au_rate,df_us],axis=1)

In [36]:
us_au_unemployment_df

,au_rate,us_rate
0,5.4,7.9
1,5.4,8.0
2,5.4,7.7
3,5.6,7.5
4,5.6,7.6
...,...,...
115,3.5,3.5
116,3.5,3.7
117,3.6,3.5
118,3.4,3.7


In [37]:
#Check if the data is valid
us_au_unemployment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   au_rate  120 non-null    float64
 1   us_rate  120 non-null    float64
dtypes: float64(2)
memory usage: 2.0 KB


In [38]:
us_au_unemployment_df['us/au unemployment ratio']=us_au_unemployment_df['us_rate']/us_au_unemployment_df['au_rate']

In [39]:
us_au_unemployment_df

,au_rate,us_rate,us/au unemployment ratio
0,5.4,7.9,1.462963
1,5.4,8.0,1.481481
2,5.4,7.7,1.425926
3,5.6,7.5,1.339286
4,5.6,7.6,1.357143
...,...,...,...
115,3.5,3.5,1.000000
116,3.5,3.7,1.057143
117,3.6,3.5,0.972222
118,3.4,3.7,1.088235


In [40]:
# Concatenating all the data into one dataframe
all_df = pd.concat([close_monthly_reset_index, us_au_unemployment_df], axis="columns", join="inner")
all_df

,date,close,au_rate,us_rate,us/au unemployment ratio
0,2012-12-31,1.046019,5.4,7.9,1.462963
1,2013-01-31,1.050035,5.4,8.0,1.481481
2,2013-02-28,1.030860,5.4,7.7,1.425926
3,2013-03-31,1.034862,5.6,7.5,1.339286
4,2013-04-30,1.037859,5.6,7.6,1.357143
...,...,...,...,...,...
115,2022-07-31,0.686085,3.5,3.5,1.000000
116,2022-08-31,0.695626,3.5,3.7,1.057143
117,2022-09-30,0.667321,3.6,3.5,0.972222
118,2022-10-31,0.636295,3.4,3.7,1.088235


In [41]:
all_df_indexed = all_df.set_index('date')
all_df_indexed

,close,au_rate,us_rate,us/au unemployment ratio
date,,,,
2012-12-31,1.046019,5.4,7.9,1.462963
2013-01-31,1.050035,5.4,8.0,1.481481
2013-02-28,1.030860,5.4,7.7,1.425926
2013-03-31,1.034862,5.6,7.5,1.339286
2013-04-30,1.037859,5.6,7.6,1.357143
...,...,...,...,...
2022-07-31,0.686085,3.5,3.5,1.000000
2022-08-31,0.695626,3.5,3.7,1.057143
2022-09-30,0.667321,3.6,3.5,0.972222


In [42]:
# Later: change the date into month

### Create the labels set (`y`)  from the “close” column, and then create the features (`X`) DataFrame from the remaining columns.

In [43]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = all_df_indexed['close']

# Separate the X variable, the features
X = all_df_indexed.copy().drop(columns = 'close')

In [44]:
# Review the y variable Series
y[0:5]

date
2012-12-31    1.046019
2013-01-31    1.050035
2013-02-28    1.030860
2013-03-31    1.034862
2013-04-30    1.037859
Name: close, dtype: float64

In [45]:
# Review the X variable DataFrame
X.head()

,au_rate,us_rate,us/au unemployment ratio
date,,,
2012-12-31,5.4,7.9,1.462963
2013-01-31,5.4,8.0,1.481481
2013-02-28,5.4,7.7,1.425926
2013-03-31,5.6,7.5,1.339286
2013-04-30,5.6,7.6,1.357143


## Split the Data into Training and Testing Sets

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [47]:
# creating a regression model
model = LinearRegression()

In [48]:
# fitting the model
model.fit(X_train, y_train)

LinearRegression()

In [49]:
# making predictions
predictions = model.predict(X_test)

In [50]:
df_predictions = pd.DataFrame(predictions)
df_predictions

,0
0,0.865361
1,0.752933
2,0.730462
3,0.689119
4,0.673673
5,0.778024
6,0.841507
7,0.666474
8,0.785395
9,0.827836


In [51]:
print(f"R-squared of training data is: {model.score(X_train, y_train)}")
print(f"R-squared of training data is: {model.score(X_test, y_test)}")

R-squared of training data is: 0.5069574695724635
R-squared of training data is: 0.28965166435572276
